In [ ]:
# Import all the things

import json
import pandas as pd
import numpy as np
import wrangle
import acquire
from prepare import basic_clean, tokenize, lemmatize, stem, remove_stopwords, prep_string_data#, split_data

import re
from sklearn.feature_extraction.text import TfidfVectorizer

from wordcloud import WordCloud
from matplotlib import pyplot as plt
import seaborn as sns

import nltk

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

# SPLITTING UP THE DATA HERE BASED ON A TSA APPROACH
### Other of our explorations might work slightly differently, if we do a prediction model or topic modeling

In [ ]:
# df = pd.read_csv('npr_corpus.csv')
# df.head()

In [ ]:
df = wrangle.get_npr_data()
df.head()

In [ ]:
# df.sort_values(by=['episode_id','episode_order'])[0:50]
df.sort_values(by=['story_id_num','utterance_order'])[0:50]

## Ok, since we already have utterance order, we can just use that as a feature
## Moving on to utterance length

In [ ]:
df.utterance[0]

In [ ]:
len(df.utterance[0])

In [ ]:
len(df.utterance[0].split())

In [ ]:
df.utterance.apply(str.split).apply(len)[0]

In [ ]:
# df['utterance_length'] = len(df.utterance.split())
df['utterance_word_count'] = df.utterance.apply(str.split).apply(len)
df.head()

#### A quick lesson on .apply:

In [ ]:
# def make_wordcloud(host):
#     etc etc

# h.apply(make_wordcloud)

# the h is what is referred to after the apply is applied

# or

# def make_wordcloud(host, ajowpj):
#     etc etc

# h.apply(make_wordcloud, extr_words= blah blah)

In [ ]:
def string_cleaning(s):
    '''
    Function to remove punctuation from word frequencies for hosts.
    '''
    # remove special characters
    s = re.sub(r"[^a-z0-9'\s\?]", '', s)

    return s

In [ ]:
df.utterance[2]

In [ ]:
# Series.str.contains(pat, case=True, flags=0, na=None, regex=True)

df.utterance.str.contains(r"[\?]", regex=True)[2]


# Ok, so this indicates thee presence of a question mark...how about a question mark count?

In [ ]:
# pandas.Series.str.count

df.utterance.str.count(r"[\?]")[2]

In [ ]:
df['question_mark_count'] = df.utterance.str.count(r"[\?]")
df.head()

## Who asks the most questions?

## Is there a difference in the mean sentiment by speaker? Program? etc
- Applied statistics-> i.e. stats testing. Is there a difference in the mean sentiment by speaker? Program? etc 

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.date = pd.to_datetime(df.date)

In [ ]:
df.dtypes

In [ ]:

# df['Date'].dt.strftime('%b-%Y')

# df['episode_date'].dt.strftime('%b')

In [ ]:
df['month']=df['date'].dt.strftime('%b')
df['year']=pd.DatetimeIndex(df['date']).year
df=df.reset_index()
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=df.set_index('date').sort_index()

In [ ]:
df = df[df.index.year >= 2005]

In [ ]:
df.head(1)

In [ ]:
train = df[:'2015']
validate = df['2016':'2017']
test = df['2018':]
train.shape, validate.shape, test.shape

In [ ]:
train.index[-1:], validate.index[:1], validate.index[-1:], test.index[:1]

In [ ]:
df.vader.hist()

## Using train split from here

In [ ]:
host_df = train[train.is_host==True]

In [ ]:
guest_df = train[train.is_host!=True]

In [ ]:
host_df.shape, guest_df.shape

In [ ]:
host_df.shape

In [ ]:
# are there repeats in the host_df? 
host_df.speaker.value_counts()

- I can see two different duplicates for steve inskeep bc of typos
- I think this is going to fall into the arena of significantly diminishing returns, and I will not address it at this time

In [ ]:
hosts_with_the_most = host_df.speaker.value_counts().head(10).index.to_list()

- This is a list of the 12 hosts with the most observations

In [ ]:
hosts_with_the_most

In [ ]:
top_hosts_df = train[train.speaker.isin(hosts_with_the_most)]


In [ ]:
top_hosts_df.head(1)

In [ ]:
top_hosts_df.index.min(), top_hosts_df.index.max()

In [ ]:
top_hosts_df.vader.mean()

- Here, we have the average sentiment score for all the top hosts; as you can see, it is relatively neutral in sentiment

In [ ]:
plt.figure(figsize=(20,10))
sns.boxplot(data=top_hosts_df, x='speaker',y='vader')
plt.xticks(rotation=90);

- The mean sentiment value is awfully close for everyone here, so i'm going to stats test it with an ANOVA test

In [ ]:
null_hypothesis = "Average sentiment score is the same across hosts"
alternative_hypothesis = "Average sentiment score is different in at least one host of the group"
alpha = 0.01 # Let's be 99% certain the result we see isn't due to chance/randomness

In [ ]:
host_df_list=[]
for host in hosts_with_the_most:
    x = host.split()
    host_df_list.append(f'{x[0]}_df')
print(host_df_list)

In [ ]:
hosts_with_the_most

In [ ]:
# Let's isolate our hosts
NEAL_df = train[train.speaker == 'neal conan, host'].vader
IRA_df = train[train.speaker ==  'ira flatow, host'].vader
ROBERT_df = train[train.speaker == 'robert siegel, host'].vader
STEVE_df = train[train.speaker == 'steve inskeep, host'].vader
MELISSA_df = train[train.speaker == 'melissa block, host'].vader
FARAI_df = train[train.speaker ==  'farai chideya, host'].vader
RENEE_df = train[train.speaker == 'renee montagne, host'].vader
SCOTT_df = train[train.speaker == 'scott simon, host'].vader
DAVID_df = train[train.speaker == 'david greene, host'].vader
RACHEL_df = train[train.speaker == 'rachel martin, host'].vader


# GUY_df = train[train.speaker == 'guy raz, host'].vader_stopped
# MADELEINE_df = train[train.speaker == 'madeleine brand, host'].vader_stopped
# MICHELE_df = train[train.speaker == 'michelle norris, host'].vader_stopped
# ALEX_df = train[train.speaker == 'alex chadwick, host'].vader_stopped


In [ ]:
# f_oneway is our ANOVA test
# See https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html for more info
from scipy.stats import f_oneway

f, p = f_oneway(NEAL_df, IRA_df, ROBERT_df, STEVE_df, MELISSA_df, FARAI_df, RENEE_df, SCOTT_df, DAVID_df, RACHEL_df)
f, p

In [ ]:
if p < alpha:
    print("We reject the null hypothesis that", null_hypothesis)
    print("We move forward with the alternative hypothesis that", alternative_hypothesis)
else:
    print("We fail to reject the null hypothesis")
    print("Evidence does not support the claim that sentiment differs from host to host")

In [ ]:
sns.catplot(x="speaker", y="vader", kind="bar", data=top_hosts_df)
plt.xticks(rotation=90);

- Another view of the same...it looks more clear here that there is a difference, however note the small scale on the y axis

In [ ]:
top_hosts_df.head(1)

### The conclusion to the above is that there is a statistically significant difference in sentiment in the top hosts group, even if it is small.

### Josh's code for the hosts' words

In [ ]:
# host_words = train[train.speaker.isin(hosts_with_the_most)].groupby('speaker')['lemmatized'].agg(lambda col: ' '.join(col))

## What's the difference in sentiment score between hosts and non-hosts?

In [ ]:
non_host_df = train[train.is_host==False]

In [ ]:
plt.figure
sns.boxplot(data=train, x='is_host',y='vader')
plt.title("The mean sentiment score for Hosts versus Non-Hosts")
plt.ylabel("Mean Sentiment Score")


In [ ]:
train[train.is_host==False].vader.mean(), train[train.is_host==True].vader.mean()

In [ ]:
null_hypothesis = "Hosts and Non-hosts have the same sentiment score"
alternative_hypothesis = "Hosts and Non-hosts have different sentiment scores"
alpha = 0.01 # we want to be 99% sure our results aren't the result of chance/randomness

In [ ]:
#stats testing on the same
# anova is the wrong test!
from scipy import stats

t, p = stats.ttest_ind(host_df.vader,non_host_df.vader)
t, p

In [ ]:
if p < alpha:
    print("We reject the null hypothesis that", null_hypothesis)
    print("We move forward with the hypothesis that", alternative_hypothesis)
else:
    print("We fail to reject the null hypothesis")
    print(f"Evidence does not support the claim that", alternate_hypothesis)

### The average sentiment score for non-hosts is somewhat higher than for the hosts.  To be expected for a relatively neutral news outlet?

## How about sentiment score by year?

In [ ]:
train.resample('Y').vader.mean()

- why don't we have vader scores in 2000-2003 on yearly resampling?

In [ ]:
vader_yearly = pd.DataFrame(train.resample('Y').vader.mean())

In [ ]:
vader_yearly.plot()

In [ ]:
vader_monthly = pd.DataFrame(train.resample('M').vader.mean())

In [ ]:
vader_monthly.head()

In [ ]:
vader_monthly.plot()

- What's with all those missing sentiment scores in the early aughts?
- We eliminated observations pre-2005, they were missing data

In [ ]:
# pd.DataFrame(df.resample("d").vader_stopped.mean()).plot()

## By day of week?

In [ ]:
train.groupby(train.index.weekday).vader.mean()
# sentiment by day, 0 is monday, 6 is sunday

In [ ]:
sns.boxplot(data=train, x=train.index.weekday,y='vader')

In [ ]:
Monday_df = train[train.index.weekday == 0]
Tuesday_df = train[train.index.weekday == 1]
Wednesday_df = train[train.index.weekday == 2]
Thursday_df = train[train.index.weekday == 3]
Friday_df = train[train.index.weekday == 4]
Saturday_df = train[train.index.weekday == 5]
Sunday_df = train[train.index.weekday == 6]

In [ ]:
null_hypothesis = "Different days of the week have the same sentiment score"
alternative_hypothesis = "At least one day of the week has a different sentiment score from the rest"
alpha = 0.01 # we want to be 99% sure our results aren't the result of chance/randomness

In [ ]:
from scipy.stats import f_oneway

f, p = f_oneway(Monday_df.vader,\
Tuesday_df.vader,\
Wednesday_df.vader,\
Thursday_df.vader,\
Friday_df.vader,\
Saturday_df.vader,\
Sunday_df.vader)

f, p

In [ ]:
if p < alpha:
    print("We reject the null hypothesis that", null_hypothesis)
    print("We move forward with the alternative hypothesis that", alternative_hypothesis)
else:
    print("We fail to reject the null hypothesis")
    print("Evidence does not support the claim that sentiment differs from one weekday to the next")

## And by program?

In [ ]:
train.program.value_counts().index.to_list()

In [ ]:
Talk_of_the_Nation_df = train[train.program == 'talk of the nation']
Morning_Edition_df = train[train.program == 'morning edition']
All_Things_Considered_df = train[train.program == 'all things considered']
News_and_Notes_df = train[train.program == 'news & notes']
Weekend_Edition_Saturday_df = train[train.program == 'weekend edition saturday']
Weekend_Edition_Sunday_df = train[train.program == 'weekend edition sunday']
Day_to_Day_df = train[train.program == 'day to day']


In [ ]:
plt.figure(figsize=(30,10))
Talk_of_the_Nation_df.resample("y").vader.mean().plot(alpha=.5)
All_Things_Considered_df.resample("y").vader.mean().plot(alpha=.5)
Morning_Edition_df.resample("y").vader.mean().plot(alpha=.5)
News_and_Notes_df.resample("y").vader.mean().plot(alpha=.5)
Day_to_Day_df.resample("y").vader.mean().plot(alpha=.5)
Weekend_Edition_Sunday_df.resample("y").vader.mean().plot(alpha=.5)
Weekend_Edition_Saturday_df.resample("y").vader.mean().plot(alpha=.5)
plt.title("Average Sentiment over Time, by Program")
plt.legend(['Talk of the Nation',
 'All Things Considered',
 'Morning Edition',
 'News & Notes',
 'Day to Day',
 'Weekend Edition Sunday',
 'Weekend Edition Saturday'], prop={'size': 20})

- Note the programs that have gone off air
- Also, there is some difference in sentiment from program to program
- Weekend Edition (Sat and Sun) have higher sentiment scores--these programs try to be a little on the lighter side for the weekend
- We're definitely starting off the day on the most pessimistic note

In [ ]:
null_hypothesis = "The different programs have the same sentiment score"
alternative_hypothesis = "At least one program has a different sentiment score"
alpha = 0.01 # we want to be 99% sure our results aren't the result of chance/randomness

In [ ]:
# f_oneway is our ANOVA test
# See https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html for more info
from scipy.stats import f_oneway

f, p = f_oneway(Talk_of_the_Nation_df.vader,
All_Things_Considered_df.vader,
Morning_Edition_df.vader,
News_and_Notes_df.vader,
Day_to_Day_df.vader,
Weekend_Edition_Sunday_df.vader,
Weekend_Edition_Saturday_df.vader,)
f, p

In [ ]:
if p < alpha:
    print("We reject the null hypothesis that", null_hypothesis)
    print("We move forward with the alternative hypothesis that", alternative_hypothesis)
else:
    print("We fail to reject the null hypothesis")
    print("Evidence does not support the claim that sentiment differs from host to host")

## How did sentiment change after 9/11 or etc...
- after obama election? 
- from start to finish of 2016 presidential campaign?

## what about clustering?
- There is very little continuous data here, clusering might not be possible.
- datetime...but how usefull can that be? 
- utterance order? is that available?
- maybe explor utterance order vs utterance length vs number of question marks or similar

In [ ]:
train.columns

In [ ]:
train.sort_values(by=['story_id_num','utterance_order'])[0:50]

### Ok, for the first time I'm seeing clearly how the episodes are in fact ordered by 'story' (episode_id) and then ordered by utterance within the story

### This is great, bc it's what is going to make topic modeling possible down the line

# First...
- Let's do a tsa model of sentiment using Prophet : )

In [ ]:
train.columns

In [ ]:
for_tsa_modeling_df = pd.DataFrame(df['vader'])

In [ ]:
for_tsa_modeling_df.head()

In [ ]:
# for_prophet_df.resample('Y').mean()
for_tsa_modeling_df.index.year.value_counts()

- there is NOTHING in there fro 2000 to 2003
- 2004 barely a thing

In [ ]:

tsa_train = for_tsa_modeling_df['2005':'2015']
tsa_validate = for_tsa_modeling_df['2016':'2017']
tsa_test = for_tsa_modeling_df['2018':]
tsa_train.shape, tsa_validate.shape, tsa_test.shape

- I've dropped all observations prior to 2005 in this split due to the problem with the data

In [ ]:
tsa_train[-1:], tsa_validate[:1], tsa_validate[-1:], tsa_test[:1]

- the splits look good

In [ ]:
# for col in prophet_train.columns:
plt.figure(figsize=(12,4))
plt.plot(tsa_train.resample('M').vader.mean())
plt.plot(tsa_validate.resample('M').vader.mean())
plt.plot(tsa_test.resample('M').vader.mean())
plt.ylabel('Sentiment Score')
plt.title('Time Series Plot of Sentiment, by Train/Validate/Test')
plt.legend(labels = ["Train","Validate","Test"])
plt.show()

In [ ]:
pd.plotting.autocorrelation_plot(tsa_train.resample('y').mean())

- There doesn't look to be any seasonality to speak of--try running the autocorrelation_plot with different resmample lengths
- from the curriculum:
    - "The dashed lines are a way measure whether the observed autocorrelation is a meaningful signal or just white noise. If the majority of the peaks and valleys fall within the dashed lines, the time series is probably white noise."

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.api import Holt

In [ ]:
# prophet_train.dropna(axis=0, how='any')
tsa_train.dropna(axis=0, how='any').resample('M').mean().plot()

In [ ]:
tsa_train[tsa_train.vader.isnull() == True]

In [ ]:
# tsa_train['vader_stopped'] = tsa_train['vader_stopped'].fillna(0)

In [ ]:
# sm.tsa.seasonal_decompose(tsa_train.resample('M').mean()).plot()


# _ = sm.tsa.seasonal_decompose(tsa_train['vader_stopped'].resample('W').mean()).plot()


# ValueError: This function does not handle missing values


### Simple Average Model

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
# Create the empty dataframe
eval_df = pd.DataFrame(columns=['model_type', 'target_var', 'rmse'])

# function to store rmse for comparison purposes
def append_eval_df(model_type, target_var):
    rmse = evaluate(target_var)
    d = {'model_type': [model_type], 'target_var': [target_var], 'rmse': [rmse]}
    d = pd.DataFrame(d)
    return eval_df.append(d, ignore_index = True)

In [ ]:
# evaluation function to compute rmse
def evaluate(target_var):
    rmse = round(sqrt(mean_squared_error(validate[target_var], yhat_df[target_var])), 4)
    return rmse

In [ ]:
score = round(tsa_train['vader'].mean(), 4)

def make_predictions():
    yhat_df = pd.DataFrame({'vader': score}, index = validate.index)

    return yhat_df

yhat_df = make_predictions()

In [ ]:
# plot and evaluate 
def plot_and_eval(target_var):
    plt.figure(figsize = (12,4))
    plt.plot(tsa_train[target_var].resample('M').mean(), label = 'Train', linewidth = 1)
    plt.plot(tsa_validate[target_var].resample('M').mean(), label = 'Validate', linewidth = 1)
    plt.plot(yhat_df[target_var])
    plt.title(target_var)
    rmse = evaluate(target_var)
    print(target_var, '-- RMSE: {:.4f}'.format(rmse))
    plt.show()

In [ ]:
# plot and evaluate 
def plot_and_eval_2(target_var,model_type):
    plt.figure(figsize = (12,4))
    plt.plot(tsa_train[target_var].resample('M').mean(), label = 'Train', linewidth = 1)
    plt.plot(tsa_validate[target_var].resample('M').mean(), label = 'Validate', linewidth = 1)
    plt.plot(yhat_df[target_var])
    plt.title(f"{target_var}, {model_type}")
    rmse = evaluate(target_var)
    print(target_var, '-- RMSE: {:.4f}'.format(rmse))
    plt.show()

In [ ]:
yhat_df.head()

In [ ]:
plot_and_eval('vader')

In [ ]:
eval_df = append_eval_df(model_type='simple_average', 
                             target_var = 'vader')

In [ ]:
eval_df

### Moving Average

In [ ]:
train['vader'].rolling(3).mean()

In [ ]:
# Trying with different periods

periods = [30, 60, 90, 365]

for p in periods:
    score = round(tsa_train['vader'].rolling(p).mean().iloc[-1], 4)
    yhat_df = make_predictions()
    model_type = str(p) + ' day moving average'
    eval_df = append_eval_df(model_type = model_type,
                             target_var = 'vader'
                            )
    plot_and_eval_2('vader', model_type)
eval_df

In [ ]:
yhat_df

# In case you missed it: there's something wrong with the moving average models.  Or am I wrong?


In [ ]:
# _ = sm.tsa.seasonal_decompose(train[col].resample('W').mean()).plot()
# plt.show()

# Still having problems loooking at seasonl decomposition due to missing valuese

In [ ]:
tsa_validate.index[-1]

In [ ]:
tsa_train['vader'].resample('M').mean()

In [ ]:
# model = Holt(tsa_train['vader'], exponential = False)
# model = model.fit(smoothing_level = .1, 
#                   smoothing_slope = .1, 
#                   optimized = False)
# yhat_items = model.predict(start = tsa_validate.index[0], 
#                            end = tsa_validate.index[-1])
# yhat_df['vader'] = round(yhat_items, 4)

# #HOLTS IS BROKEN : (

# # KeyError: 'The `start` argument could not be matched to a location related to the index of the data.'


In [ ]:
# yhat_df.head()
# the above is returning NaNs only

In [ ]:
# plot_and_eval('vader')

### Moving prophet model to its own notebook bc of the size of it (not pushing to github)

## The confidence intervals on the Prophet model were massive.  The sentiment observations in this data are very close together and yield rather little predictive power

In [ ]:
tsa_train.groupby([tsa_train.index.year, tsa_train.index.month]).mean().unstack(0).plot()
plt.legend(loc=(1.3,.5), bbox_to_anchor=(0.5, 0., 0.5, 0.5))

- Is there a trend of dipping sentiment in the summer?
- Also, it seems like the vader scores are lower in the later years

In [ ]:
train.head()

In [ ]:
train.dtypes

# MODELING

### BASELINE:


In [ ]:
# THIS WAS HANDY CODE TO ELIMINATE SPEAKERS WITH ONLY 1 OR 2 OBSERVATIONS
# THOSE SPEAKERS WERE CREATING PROBLEMS WITH SPLITTING
# df.groupby('speaker').filter(lambda x : len(x)<=2) 

## In the next cell, i've sampled down the original df and derived all the splits based on 'rest', which was based on the small_df

In [ ]:
small_df = df.sample(10_000)

counts = small_df['speaker'].value_counts()

rest = small_df[~small_df['speaker'].isin(counts[counts < 3].index)]
# df[df.groupby('speaker').filter(lambda x : len(x)<=2)] #this code doesn't work

# res works perfectly : ))))))))))))))

In [ ]:
rest

In [ ]:
df.shape,rest.shape

In [ ]:
rest.isna().sum()

In [ ]:
rest = rest.dropna()

# Trying a new approach to splitting:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer()
X = tfidf.fit_transform(rest.lemmatized) # is this a problem? to fit on the whole df? 
#instead of just X? Been meaning to ask a while, this is from lesson
y = rest.is_host

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.2)

train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

lm = LogisticRegression().fit(X_train, y_train)

train['predicted'] = lm.predict(X_train)
test['predicted'] = lm.predict(X_test)

## Thank god, we have a model.  Built on TF-IDF and Logistic Regression

In [ ]:
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.predicted, train.actual))
print('---')
print(classification_report(train.actual, train.predicted))

In [ ]:
# classification_report(train.actual, train.predicted)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
X= cv.fit_transform(rest.lemmatized)
y = rest.is_host

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.2)

train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

lm = LogisticRegression().fit(X_train, y_train)

train['predicted'] = lm.predict(X_train)
test['predicted'] = lm.predict(X_test)

In [ ]:
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.predicted, train.actual))
print('---')
print(classification_report(train.actual, train.predicted))